In [1]:
!pip install gmdh
!pip install numpy==1.24.4

In [12]:
# Импортируем необходимые библиотеки
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Модели
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, StackingRegressor
from sklearn.linear_model import Ridge, LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.neural_network import MLPRegressor

# GMDH (COMBI и Multi)
from gmdh import Combi, Multi, split_data, Mia

In [3]:
# Загрузка датасета из KaggleHub
import kagglehub

# Скачиваем датасет
path = kagglehub.dataset_download("niteshyadav3103/concrete-compressive-strength")
print("Path to dataset files:", path)

# Читаем данные
data = pd.read_csv(f"{path}/Concrete Compressive Strength.csv")

# Быстрый просмотр данных
data.head()

<frozen importlib._bootstrap>:1047: ImportWarning: _PyDriveImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:1047: ImportWarning: _BokehImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:1047: ImportWarning: _PyDriveImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:1047: ImportWarning: _BokehImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:1047: ImportWarning: _PyDriveImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:1047: ImportWarning: _BokehImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:1047: ImportWarning: _PyDriveImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:1047: ImportWarning: _BokehImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._boots

100%|██████████| 14.1k/14.1k [00:00<00:00, 8.72MB/s]

Extracting files...
Path to dataset files: /root/.cache/kagglehub/datasets/niteshyadav3103/concrete-compressive-strength/versions/2



<frozen importlib._bootstrap>:1047: ImportWarning: _PyDriveImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:1047: ImportWarning: _BokehImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:1047: ImportWarning: _PyDriveImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:1047: ImportWarning: _BokehImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:1047: ImportWarning: _PyDriveImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:1047: ImportWarning: _BokehImportHook.find_spec() not found; falling back to find_module()


,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age (day),Concrete compressive strength
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28,79.986111
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28,61.887366
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270,40.269535
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365,41.052780
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360,44.296075


In [4]:
# Проверка на пропущенные значения
print(data.isnull().sum()) # все ок

# Переименуем целевую переменную для удобства
data.columns = data.columns.str.strip()  # Убираем лишние пробелы
data.rename(columns={'Concrete compressive strength': 'strength'}, inplace=True)

# Проверим, что название обновилось
print(data.columns)

# Разделяем данные на признаки и целевую переменную
X = data.drop(columns=['strength'])
y = data['strength']

# Делим на обучающую и тестовую выборку
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Масштабируем числовые данные
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

Cement                            0
Blast Furnace Slag                0
Fly Ash                           0
Water                             0
Superplasticizer                  0
Coarse Aggregate                  0
Fine Aggregate                    0
Age (day)                         0
Concrete compressive strength     0
dtype: int64
Index(['Cement', 'Blast Furnace Slag', 'Fly Ash', 'Water', 'Superplasticizer',
       'Coarse Aggregate', 'Fine Aggregate', 'Age (day)', 'strength'],
      dtype='object')


In [5]:
# Определяем базовые модели
base_models = [
    ('lr', LinearRegression()),
    ('dt', DecisionTreeRegressor()),
    ('rf', RandomForestRegressor(n_estimators=100, random_state=42))
]

# Определяем мета-оценщик
final_estimator = GradientBoostingRegressor(n_estimators=100, random_state=42)

# Создаем стекинг-модель – объединяет три базовые модели (линейная регрессия, дерево решений и случайный лес) и финальный мета-оценщик (градиентный бустинг)
stacking_model = StackingRegressor(estimators=base_models, final_estimator=final_estimator)

# Обучаем модель
stacking_model.fit(X_train, y_train)

# Предсказываем результаты
y_pred = stacking_model.predict(X_test)

# Оцениваем модель
print(f"Mean Squared Error (MSE): {mean_squared_error(y_test, y_pred)}")
print(f"R^2 Score: {r2_score(y_test, y_pred)}")

Mean Squared Error (MSE): 29.265861584512397
R^2 Score: 0.8918395043509553


* MLP — это сеть из нескольких слоев нейронов (искусственных «узлов»), которые связаны между собой.
* Есть **входной слой** (получает данные),
* Один или несколько **скрытых слоев** (обрабатывают данные с помощью математических функций),
* И **выходной слой** (даёт результат — предсказание).

### Как работает MLP:

1. Каждый нейрон в слое получает числовые данные (входы).
2. Он умножает эти данные на веса, складывает с некоторым смещением (bias).
3. Пропускает сумму через функцию активации (например, ReLU или сигмоида), чтобы добавить нелинейность.
4. Результат передаётся дальше на следующий слой.
5. На выходе получается итоговое значение (например, предсказанная величина).

In [6]:
# Инициализация и обучение MLP
mlp_model = MLPRegressor(hidden_layer_sizes=(100, 100), activation='relu', solver='adam', max_iter=1000, random_state=42)
mlp_model.fit(X_train, y_train)

# Предсказания
y_pred_mlp = mlp_model.predict(X_test)

# Оценка модели
print(f"Mean Squared Error (MSE): {mean_squared_error(y_test, y_pred_mlp)}")
print(f"R^2 Score: {r2_score(y_test, y_pred_mlp)}")

Mean Squared Error (MSE): 27.380981471348207
R^2 Score: 0.8988056265233759


In [15]:
# Восстанавливаем данные
X = data.drop(columns=['strength'])
y = data['strength']

# Делим данные на обучающую и тестовую выборку
x_train, x_test, y_train, y_test = split_data(X.values, y.values)

# Нормализуем признаки (важно для моделей GMDH)
scaler = StandardScaler()
x_train_scaled = scaler.fit_transform(x_train)
x_test_scaled = scaler.transform(x_test)

# Обучение модели COMBI
combi_model = Combi()
combi_model.fit(x_train_scaled, y_train)

# Предсказание COMBI
y_pred_combi = combi_model.predict(x_test_scaled)

# Оценка COMBI
print(f"COMBI - Mean Squared Error (MSE): {mean_squared_error(y_test, y_pred_combi):.4f}")
print(f"COMBI - R^2 Score: {r2_score(y_test, y_pred_combi):.4f}")

# Обучение модели MIA
mia_model = Mia()
mia_model.fit(x_train_scaled, y_train)

# Предсказание MIA
y_pred_mia = mia_model.predict(x_test_scaled)

# Оценка MIA
print(f"MIA - Mean Squared Error (MSE): {mean_squared_error(y_test, y_pred_mia):.4f}")
print(f"MIA - R^2 Score: {r2_score(y_test, y_pred_mia):.4f}")

COMBI - Mean Squared Error (MSE): 240.2948
COMBI - R^2 Score: -0.5743
MIA - Mean Squared Error (MSE): 226.3270
MIA - R^2 Score: -0.4828
